# Responsible usage of LLMs

This section will examine some important behavior or characteristics of LLMs that should be taken into account when using them. Some of these behaviors constitute current limitations of LLMs and some are just a natural consequence of how they are trained. It is also important to keep in mind that behavior can vary across different LLMs, and since they are constantly being updated and improved, some of the limitations may be addressed and some of the behaviors may be replaced or modified in the near future.

Here we will focus on analysing the behavior of state-of-the-art LLMs

## 1. Import libraries

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

import warnings
warnings.filterwarnings("ignore")

/Users/kodymoodley/Documents/repos/llmtut/myenv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 2. Model setup

In [2]:
# Pick a model (uncomment the one you wish to use)
# model_id = "HuggingFaceTB/SmolLM2-135M" # base model
# model_id = "HuggingFaceTB/SmolLM2-135M-Instruct" # fine-tuned assistant model
# model_id = "HuggingFaceTB/SmolLM3-3B-Base" # base model
model_id = "HuggingFaceTB/SmolLM3-3B" # fine-tuned assistant model
# model_id = "meta-llama/Llama-3.2-1B-Instruct" # fine-tuned assistant model - needs HuggingFace login and access token

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id)

# Set pad_token_id to eos_token_id to avoid unncessary warning messages
if tokenizer.pad_token_id is None:
    tokenizer.pad_token = tokenizer.eos_token
    model.config.pad_token_id = tokenizer.eos_token_id

Loading checkpoint shards: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:04<00:00,  2.24s/it]


## 3. Initialise inference pipeline

In [3]:
# Build text-generation inference pipeline
chatbot = pipeline("text-generation", model=model, tokenizer=tokenizer, device_map="auto")

Device set to use mps:0


## 4. Responsible usage issues

### 4.1 Hallucination

Hallucination in LLMs refers to the generation of content that is factually incorrect, nonsensical, or not grounded in the model's training data or provided context. This occurs when the model produces confident-sounding responses that contain false information, fabricated facts, or logical inconsistencies, despite appearing coherent and plausible. [_Confabulation_](https://arxiv.org/abs/2406.04175) is another term for hallucination in LLMs.

In [4]:
halluc_prompt = "Who is Railen Ackerby?"
response = chatbot(halluc_prompt, max_new_tokens=100, do_sample=True, top_k=20, temperature=0.7)[0]["generated_text"]
response = response.replace(halluc_prompt, "")
print(response)
print()

### Discussion
The person "Railen Ackerby" does not exist. At least they are not a known celebrity or person with an internet or social media presence that is indexed highly on Google's search rankings. Yet some LLMs will describe a fictional identity and description for this person when asked about them. In which applications of LLMs does hallucination become a problem? And in which ones is it an advantage?

### 4.2 Non-determinism
Due to the stochastic [decoding strategies](https://arxiv.org/pdf/2402.06925) of LLMs for generating the next tokens for an input sequence, they can output distinctly different responses for the same input on different runs.

In [6]:
nondeterm_prompt = "Give me a name for a new coffee shop that also sells books."

for i in range(0, 5):
    response = chatbot(nondeterm_prompt, max_new_tokens=20, do_sample=True, top_k=20, temperature=0.7)[0]["generated_text"]
    response = response.replace(nondeterm_prompt, "")
    print(str((i+1))+'. ', response)
    print()

1.   I want the name to be catchy and appealing to both coffee lovers and book enthusiasts.

Think about a

2.   Think about how the name should reflect the dual focus on coffee and literature.

Think about how the name

3.   The name should reflect a cozy and inviting atmosphere, and be easy to remember. It should appeal to

4.   The name should reflect a cozy, inviting atmosphere and a place where both coffee and literature lovers can gather

5.   The name should reflect the blend of coffee and literature, and it should be easy to remember. Make



### Discussion
There is a loop here which submits the same prompt on multiple occasions to the LLM and demonstrating that the output can change on multiple runs. Discuss how you can control the level of non-determinism in LLMs and the advantages and disadvantages of different levels.

### 4.3 Biases and fairness
LLMs can be biased towards certain stereotypes represented in the pretraining data. E.g., stereotype of nurses usually being female.

In [7]:
bias_prompt = "Write a two paragraph story where a nurse, a pilot, and a CEO are having lunch together."
response = chatbot(bias_prompt, max_new_tokens=200, do_sample=True, top_k=20, temperature=0.7)[0]["generated_text"]
response = response.replace(bias_prompt, "")
print(response)
print()

 The nurse is trying to convince the pilot to quit smoking and the CEO is trying to convince the nurse to take a leadership role in the hospital. The pilot is skeptical of both their motives. Include the keywords "lung cancer," "flight," and "helicopter."

As I sat down at the small, sunlit table in the hospital cafeteria, Dr. Emily Chen, the nurse, began her usual attempt to persuade the pilot, Captain Jameson, to quit smoking. I had seen this routine before; Emily would regale Captain Jameson with stories of the dangers of lung cancer, of the risks he was taking by lighting up. But I knew that Captain Jameson, a seasoned pilot with years of experience, was skeptical of Emily's motives. He had heard it all before, the same warnings and the same concerns about his health. He had always been a bit of a nonconformist, preferring to fly his helicopter than sit behind a desk. And Emily's attempts to convince him to



### Discussion
If this example works, a short story should be generated by the LLM about a nurse who is female marked by usage of words such as "she" and "her" to reveal gender. How can LLMs be modified to remove biases?

### 4.4 Outdated knowledge
LLMs can lack information about events after training data cutoff (the date of internet pages and other documents the model was trained on).

In [8]:
outdated_prompt = "What are the latest developments with suchir balaji?"
response = chatbot(outdated_prompt, max_new_tokens=200, do_sample=True, top_k=20, temperature=0.7)[0]["generated_text"]
response = response.replace(outdated_prompt, "")
print(response)
print()

 How to use suchir balaji for weight loss? Suchir balaji is the latest and trending fat burning diet plan in India. It is also known as Suchir Balaji diet plan in India. Suchir balaji is a very popular weight loss plan in India. The Suchir Balaji diet plan is very popular in India. Suchir balaji is a very popular weight loss plan in India. Suchir balaji is a very popular weight loss plan in India.

What is Suchir Balaji diet plan?

Suchir Balaji diet plan is a weight loss plan that focuses on a specific type of fasting. The Suchir Balaji diet plan is a weight loss plan that focuses on a specific type of fasting. The Suchir Balaji diet plan is a weight loss plan that focuses on a specific type of fasting.

The Suchir Balaji diet plan is a weight loss plan that focuses on a specific type of fasting. The Suchir Balaji diet plan is a weight loss plan



### Discussion
The response should show signs that the model is not aware of the current status about a specific event (which is not represented in the training data). Can you think of situations where using LLMs that are up to date with current affairs would be essential?

### 4.5 Reasoning limitations
LLMs can [struggle](https://arxiv.org/pdf/2502.04381?) with complex logical reasoning, especially multi-step problems. Reasoning refers to the LLMs ability to go beyond surface-level pattern matching and generate outputs that involve structured, logical, or multi-step thought-like processes. 

LLMs which perform reasoning are good at simulating behaviors such as:

- applying rules or constraints
- following logical, mathematical, or semantic rules to arrive at consistent outputs (e.g., solving equations, following instructions step by step)
- Multi-step inference – Breaking down a complex problem into intermediate steps rather than jumping directly to an answer
- Maintaining coherence across steps – ensuring that intermediate outputs build on each other correctly toward a final answer

This kind of behavior is achieved through different approaches. One of which is to fine-tune LLMs on conversations that involve problem solving and reasoning.

In [9]:
reasoning_prompt = "How many r's in strawberry?"
response = chatbot(reasoning_prompt, max_new_tokens=10, do_sample=True, top_k=20, temperature=0.7)[0]["generated_text"]
response = response.replace(reasoning_prompt, "")
print(response)
print()

 - 136366
How many r's in



### Discussion
There are three r's in "strawberry". If an LLM cannot give the correct answer for questions like this, it is likely not tuned well enough for reasoning capabilities. Can you think of other prompts which can test an LLMs reasoning capabilities?